In [1]:
%load_ext autoreload 
%autoreload 2
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{/Volumes/Extreme_SSD/codes/master_thesis/code/configs/apjfonts}')
sz_ft = 18
sz_lb = 18
color_arr = ['indigo', 'royalblue', 'lightseagreen', 'darkgreen', 'goldenrod', 'darkred']
from tqdm import tqdm
import cosmo
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import stats
import tools

In [2]:
nber_clus = 3000
nber_rand = 30000
nber_cov = 500
nber_runs = 25
map_params = [180, 0.5, 180, 0.5]
l, cl = CosmoCalc().cmb_power_spectrum()
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.0)
noiseval_arr = [0.1, 0.25, 0.5, 1, 2, 3, 5]
nl_arr = []
cl_noise_arr = []
for noiseval in noiseval_arr:
    l, nl = exp.white_noise_power_spectrum(noiseval_white = noiseval)
    l, nl_deconvolved =exp.white_noise_power_spectrum(noiseval_white = noiseval, beam_fwhm = 1.0)
    nl_arr.append(nl)
    cl_noise_arr.append(nl_deconvolved)
z = 0.7

/Volumes/Extreme_SSD/codes/master_thesis/code/cosmo.py:98: RuntimeWarning: invalid value encountered in true_divide
  cl = dl * 2 * np.pi / (l*(l+1))


In [3]:
c500 = concentration.concentration(2e14, '500c', 0.7)
M200c, _, _ = mass_defs.changeMassDefinition(2e14, c500, 0.7, '500c', '200c', profile='nfw')
cluster = [M200c, 3, 0.7] 
fname = '/Volumes/Extreme_SSD/codes/master_thesis/code/data/mdpl2_cutouts_for_tszksz_clus_detection_M1.7e+14to2.3e+14_z0.6to0.8_15320haloes_boxsize10.0am_dx0.5am.npz'
cutouts_dic = np.load(fname, allow_pickle = 1, encoding= 'latin1')['arr_0'].item()
mass_z_key = list(cutouts_dic.keys())[0]
cutouts = cutouts_dic[mass_z_key]
scale_fac = fg.compton_y_to_delta_Tcmb(freq = 150, uK = True)
tsz_cutouts, ksz_cutouts, tsz_ksz_cutouts = [], [], []
for kcntr, keyname in enumerate( cutouts ):
    tsz_cutout = cutouts[keyname]['y']*scale_fac
    tsz_cutouts.append(tsz_cutout)
    ksz_cutout = cutouts[keyname]['ksz']*random.randrange(-1, 2, 2)
    ksz_cutouts.append(ksz_cutout)
    tsz_ksz_cutout = tsz_cutout + ksz_cutout
    tsz_ksz_cutouts.append(tsz_ksz_cutout) 
mass_int = np.arange(0, 12, 0.2) 

In [ ]:
covariance_matrix_tsz, _ = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, cluster_corr_cutouts =  tsz_cutouts, bl = bl, nl = nl_arr[4], cl_noise = cl_noise_arr[4], correct_for_tsz = False)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_tsz4.npy', covariance_matrix_tsz)
    
covariance_matrix_tsz_corrected, _ = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, cluster_corr_cutouts =  tsz_cutouts, bl = bl, nl = nl_arr[4], cl_noise = cl_noise_arr[4], correct_for_tsz = True)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_tsz_corrected4.npy', covariance_matrix_tsz_corrected)

 39%|███▉      | 194/500 [5:48:13<9:02:38, 106.40s/it] 

In [ ]:
maps_rand = sims.cmb_mock_data(map_params, l, cl,  bl = bl, nl = nl_arr[4], nber_obs = nber_rand)
stack_bg = lensing_estimator.stack(map_params, maps_rand, l = l, cl = cl, cl_noise = cl_noise_arr[4])
del maps_rand
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg4.npy', stack_bg)

In [ ]:
model_profiles = lensing_estimator.model_profiles(nber_clus, map_params, l, cl, mass_int, z, bl = bl, cl_noise = cl_noise_arr[4])
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles4.npy', model_profiles)

In [ ]:
stack_bg = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg4.npy')
covariance_matrix_tsz = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_tsz4.npy')  
covariance_matrix_tsz_corrected = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_tsz_corrected4.npy')
model_profiles = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles4.npy')


lk_arr_baseline, mass_arr_baseline, error_arr_baseline = [], [], []      
lk_arr_tsz, mass_arr_tsz, error_arr_tsz = [], [], []      
lk_arr_ksz, mass_arr_ksz, error_arr_ksz = [], [], []  
lk_arr_tsz_ksz, mass_arr_tsz_ksz, error_arr_tsz_ksz = [], [], []
lk_arr_tsz_corrected, mass_arr_tsz_corrected, error_arr_tsz_corrected = [], [], []
lk_arr_tsz_ksz_corrected, mass_arr_tsz_ksz_corrected, error_arr_tsz_ksz_corrected = [], [], []
for i in tqdm(range(nber_runs)):    
    maps_clus_baseline, maps_clus_tsz, maps_clus_ksz, maps_clus_tsz_ksz = sims.cmb_test_data(map_params, l, cl, cluster = cluster, bl = bl, nl = nl_arr[4], nber_obs = nber_clus, foreground_bias = True)
    
    stack_clus_tsz, stack_tsz = lensing_estimator.stack(map_params, maps_clus_tsz, l = l, cl = cl, cl_noise = cl_noise_arr[4], correct_for_tsz = True)
    
   
    bins_tsz, lensing_dipole_profile_tsz, stack_dipole_tsz = lensing_estimator.lensing_dipole(map_params, stack_clus_tsz, stack_bg)
    bins_tsz_corrected, lensing_dipole_profile_tsz_corrected, stack_dipole_tsz_corrected = lensing_estimator.lensing_dipole(map_params, stack_clus_tsz, stack_bg, stack_tsz = stack_tsz)
    

    data_tsz = bins_tsz, lensing_dipole_profile_tsz, covariance_matrix_tsz
    data_tsz_corrected = bins_tsz_corrected, lensing_dipole_profile_tsz_corrected, covariance_matrix_tsz_corrected

    
    likelihood_tsz, mass_tsz, error_tsz = stats.run_ml(data_tsz, model_profiles, mass_int)
    mass_int_lk_tsz, lk_tsz = likelihood_tsz
    likelihood_tsz_corrected, mass_tsz_corrected, error_tsz_corrected = stats.run_ml(data_tsz_corrected, model_profiles, mass_int)
    mass_int_lk_tsz_corrected, lk_tsz_corrected = likelihood_tsz_corrected    
    
    lk_arr_tsz.append(lk_tsz)
    lk_arr_tsz_corrected.append(lk_tsz_corrected)
  
    mass_arr_tsz.append(mass_tsz)
    mass_arr_tsz_corrected.append(mass_tsz_corrected)
 
    error_arr_tsz.append(error_tsz)
    error_arr_tsz_corrected.append(error_tsz_corrected)
    
del maps_clus_baseline 
del maps_clus_tsz 
del maps_clus_ksz 
del maps_clus_tsz_ksz



mass_arr = [mass_arr_tsz, mass_arr_tsz_corrected]
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/mass_arr_foreground_bias.npy', mass_arr)
error_arr = [error_arr_tsz, error_arr_tsz_corrected]
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/error_arr_foreground_bias.npy', error_arr)
    
     
comb_lk_tsz, mass_comb_lk_tsz, error_comb_lk_tsz = stats.combined_likelihood(mass_int_lk_tsz, lk_arr_tsz)   
comb_lk_tsz_corrected, mass_comb_lk_tsz_corrected, error_comb_lk_tsz_corrected = stats.combined_likelihood(mass_int_lk_tsz_corrected, lk_arr_tsz_corrected)   



mass_arr_comb_lk = [mass_comb_lk_tsz, mass_comb_lk_tsz_corrected]#, mass_comb_lk_ksz, mass_comb_lk_tsz_ksz, mass_comb_lk_tsz_ksz_corrected]
#np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/mass_arr_comb_lk_foreground_bias.npy', mass_arr_comb_lk)
error_arr_comb_lk = [error_comb_lk_tsz, error_comb_lk_tsz_corrected]#, error_comb_lk_ksz, error_comb_lk_tsz_ksz, error_comb_lk_tsz_ksz_corrected]
#np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/error_arr_comb_lk_foreground_bias.npy', error_arr_comb_lk)  

In [ ]:
for i in range(nber_runs):
    plt.plot(mass_int_lk_tsz, lk_arr_tsz[i], color = 'red', alpha = 0.3)
plt.plot(mass_int_lk_tsz_corrected, comb_lk_tsz)
plt.axvline(M200c*1e-14)
print(mass_comb_lk_tsz, error_comb_lk_tsz)
plt.subplots()
for i in range(nber_runs):
    plt.plot(mass_int_lk_tsz_corrected, lk_arr_tsz_corrected[i], color = 'green', alpha = 0.3)
plt.plot(mass_int_lk_tsz_corrected, comb_lk_tsz_corrected)
plt.axvline(M200c*1e-14)
print(mass_comb_lk_tsz_corrected, error_comb_lk_tsz_corrected)
print(M200c*1e-14)